In [1]:
import pandas as pd
import gensim

In [2]:
IN_DATA_DIR = "../../data/raw/"
INPUT_FILE_NAME = 'cleaned.parquet'
OUT_DATA_DIR = "../../data/processed/"

In [3]:
df = pd.read_parquet(IN_DATA_DIR + INPUT_FILE_NAME)
df.head()
len(df)

2386

In [4]:
def count_tags(tag_column):
    tags = tag_column.str.replace(', ', ',').str.lower().str.strip()
    joined_tags = tags.str.cat(sep=',').split(',')
    all_tags_w_dup = pd.Series(joined_tags)

    tag_counts = all_tags_w_dup.value_counts()
    tag_list = list(tag_counts.index)
    return tag_counts, tag_list

In [5]:
# split_tags = tags.str.split(',')
# tag_counts_per_talk = split_tags.apply(len)

tags, tag_list = count_tags(df['tags'])

In [6]:
#remove ted tags
def remove_ted_tags(tags,tag_list):
    ted_tags=[]
    for tag in tag_list:
        if 'ted' in tag:
            ted_tags.append(tag)
    print(ted_tags)
    tags = tags.drop(labels = ted_tags)
    tag_list = list(tags.index.values)
    return tags, tag_list, ted_tags

def remove_tags(original_tags,remove_list):
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    
    final_tags = []
    for tag in split_tags:
        if tag not in remove_list:
            
            final_tags.append(tag)
    final_string = ','.join(final_tags)
    return final_string

In [7]:
tags, tag_list, ted_tags = remove_ted_tags(tags,tag_list)

['tedx', 'ted fellows', 'ted brain trust', 'ted prize', 'united states', 'tedyouth', 'tedmed', 'ted books', 'ted residency', 'augmented reality', 'ted en español', 'ted en espanol', 'ted-ed']


In [8]:
tags


technology         695
science            522
global issues      483
culture            470
design             400
business           329
entertainment      285
health             226
innovation         212
education          206
art                204
society            202
social change      198
communication      185
politics           183
future             181
creativity         174
biology            174
humanity           164
collaboration      163
environment        155
economics          154
medicine           154
brain              148
activism           147
invention          136
community          136
children           135
history            135
health care        130
                  ... 
sleep                3
microsoft            3
trust                3
ants                 3
ptsd                 3
pandemic             3
apes                 3
moon                 2
cello                2
driverless cars      2
grammar              2
anthropocene         2
arts       

In [9]:
def squashing (x,squash_list):
    original_tags = x
#     print(x)
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    final_tags = []
    for tag in split_tags:
        if tag in squash_list:
            
            final_tags.append(tag)
    final_string = ','.join(final_tags)
    return final_string

In [10]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [11]:
nlp_list = []
for i in range(len(tag_list)):
    nlp_list.append(nlp(tag_list[i]))
# print(nlp_list)

In [12]:
simlist = []
tagsimcount = [0]*len(tag_list)
tagsimdict = {el:[] for el in tag_list}
for outer in range(len(nlp_list)):
    for inner in range(len(nlp_list)):
        if inner<outer:
            sim = nlp_list[outer].similarity(nlp_list[inner])
            if sim > 0.65:
                ow = tag_list[outer]
                iw = tag_list[inner]
                ow_count = tags.at[ow]
                iw_count = tags.at[iw]
                simlist.append([ow,iw,sim])
                tagsimcount[outer] += 1
                tagsimcount[inner] += 1
                tagsimdict[ow].append(iw)
                tagsimdict[iw].append(ow)
                
items =list(tagsimdict.items())

for k,v in items:
    if v == []:
        del tagsimdict[k]             

# print(simlist)
# print(tagsimcount)
# print(tagsimdict)
sim_tags = list(tagsimdict.keys())

C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWa

C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWa

C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWa

C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWa

C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWa

...

C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWa

C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWa

C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWa

C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWa

C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
C:\Users\JSaw\Anaconda3\lib\runpy.py:193: UserWa

In [13]:
# replace tags
# GENERALISE: 'education','teaching' -> 'education'
# REMAIN: 'africa', 'asia' -> 'africa', 'asia'

#anything with ted remove hahaha
print(tagsimcount)
print(tagsimdict)

[1, 7, 3, 2, 3, 2, 0, 5, 3, 3, 6, 3, 6, 0, 2, 1, 1, 12, 3, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 5, 3, 2, 0, 0, 1, 1, 0, 1, 0, 0, 0, 7, 2, 3, 1, 0, 5, 0, 1, 7, 2, 2, 5, 2, 2, 0, 1, 0, 0, 0, 3, 0, 0, 1, 0, 1, 3, 1, 1, 3, 1, 2, 1, 1, 0, 1, 0, 1, 2, 3, 7, 6, 3, 0, 1, 0, 0, 7, 0, 5, 3, 0, 0, 4, 1, 3, 0, 2, 0, 0, 2, 7, 0, 1, 1, 3, 2, 1, 0, 1, 1, 0, 2, 1, 0, 3, 1, 0, 1, 1, 1, 0, 0, 7, 0, 0, 0, 0, 0, 0, 1, 2, 0, 4, 0, 2, 3, 0, 1, 0, 3, 3, 3, 1, 1, 0, 7, 1, 0, 1, 1, 0, 1, 0, 0, 4, 0, 0, 0, 1, 3, 0, 1, 2, 5, 3, 1, 1, 0, 5, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 4, 0, 0, 4, 1, 0, 0, 1, 0, 0, 0, 5, 3, 2, 4, 0, 0, 4, 4, 1, 0, 0, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 1, 2, 1, 1, 0, 4, 0, 2, 0, 0, 3, 1, 2, 6, 1, 0, 0, 0, 2, 1, 0, 0, 4, 0, 0, 0, 1, 3, 2, 0, 3, 0, 0, 2, 0, 0, 6, 0, 0, 1, 0, 2, 0, 2, 1, 4, 0, 0, 1, 0, 0, 0, 0, 2, 0, 2, 1, 0, 0, 2, 1, 1, 1, 0, 0, 1, 0, 0, 1, 5, 2, 0, 0, 0, 0, 1, 3, 1, 1, 2, 0, 0, 2, 0, 1, 3, 0, 1, 5, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 2, 2, 0, 3,

In [14]:
def sim_counts(a):
#     print(a)
    a_count=tags.at[a]
    all_tags = [a]
    all_count = [a_count]
    b = tagsimdict[a]
    for word in b:
        b_count=tags.at[word]
        all_tags.append(word)
        all_count.append(b_count)
    max_count = max(all_count)
    max_index = all_count.index(max_count)
    major = all_tags[max_index]
    minor = all_tags
    minor.remove(major)
    final = [major]+minor
    return final

In [15]:
sim_tags_ordered = []
for tag in tag_list:
    if tag in sim_tags:
        sim_tags_ordered.append(tag)
# print(sim_tags_ordered)
sim_tags_reversed = sim_tags_ordered
sim_tags_reversed.reverse()
print(sim_tags_reversed)
print(len(sim_tags),len(sim_tags_reversed))

['gay', 'syria', 'arts', 'grammar', 'driverless cars', 'cello', 'apes', 'ants', 'movies', 'street art', 'sexual violence', 'criminal justice', 'manufacturing', 'friendship', 'jazz', 'nuclear weapons', 'development', 'funny', 'primates', 'violin', 'resources', 'iran', 'state-building', 'mindfulness', 'atheism', 'south america', 'christianity', 'wind energy', 'student', 'gender spectrum', 'geology', 'monkeys', 'extreme sports', 'painting', 'transgender', 'meditation', 'aircraft', 'latin america', 'debate', 'rocket science', 'buddhism', 'foreign policy', 'men', 'charter for compassion', 'suicide', 'archaeology', 'farming', 'islam', 'nuclear energy', 'advertising', 'vocals', 'big bang', 'autism spectrum disorder', 'urban', 'natural disaster', 'brazil', 'capitalism', 'paleontology', 'medical imaging', 'human origins', 'obesity', 'birds', 'bees', 'body language', 'solar energy', 'extraterrestrial life', 'solar', 'iraq', 'piano', 'lgbt', 'gender equality', 'microbes', 'synthetic biology', 'co

In [16]:
final_replacement = {}
for i in range(len(sim_tags_reversed)):
    tag = sim_tags_reversed[i]
    sims = tagsimdict[tag]
    #check if words that main tag is similar too have similar words too
    for sim in sims:
        if sim in final_replacement.keys():
            simlist = final_replacement[sim]
            sims+=simlist
            del final_replacement[sim]
    #remove duplicates
    sims = list(dict.fromkeys(sims))
    final_replacement[tag]=sims
print(final_replacement)

{'monkeys': ['primates', 'apes', 'monkeys'], 'meditation': ['mindfulness', 'meditation'], 'buddhism': ['christianity', 'religion', 'islam', 'buddhism', 'atheism'], 'autism spectrum disorder': ['gender spectrum', 'gender', 'gender equality', 'autism spectrum disorder'], 'brazil': ['latin america', 'africa', 'asia', 'europe', 'brazil', 'south america', 'middle east'], 'paleontology': ['archaeology', 'geology', 'anthropology', 'paleontology'], 'iraq': ['iran', 'syria', 'iraq', 'egypt'], 'lgbt': ['transgender', 'lgbt'], 'egypt': ['syria'], 'guitar': ['piano', 'vocals', 'violin', 'guitar', 'cello'], 'natural resources': ['natural disaster', 'resources', 'disaster relief', 'natural resources'], 'flight': ['aircraft', 'flight'], 'solar system': ['solar', 'solar energy', 'wind energy', 'solar system', 'energy', 'alternative energy', 'nuclear energy', 'nuclear weapons'], 'cars': ['driverless cars', 'cars'], 'consumerism': ['capitalism', 'democracy', 'consumerism'], 'disaster relief': ['natural 

In [17]:
print(final_replacement.keys())

dict_keys(['monkeys', 'meditation', 'buddhism', 'autism spectrum disorder', 'brazil', 'paleontology', 'iraq', 'lgbt', 'egypt', 'guitar', 'natural resources', 'flight', 'solar system', 'cars', 'consumerism', 'disaster relief', 'bacteria', 'urban planning', 'gender', 'biomechanics', 'plants', 'feminism', 'investment', 'sports', 'middle east', 'insects', 'agriculture', 'goal-setting', 'sex', 'death', 'law', 'relationships', 'universe', 'energy', 'religion', 'language', 'biodiversity', 'writing', 'big problems', 'violence', 'film', 'media', 'work', 'internet', 'government', 'inequality', 'africa', 'life', 'humor', 'war', 'animals', 'women', 'music', 'history', 'humanity', 'future', 'politics', 'business', 'design', 'culture', 'global issues', 'science', 'technology'])


In [18]:
def replacing (x):
    original_tags = x
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    final_tags = []
    for tag in split_tags:
        if tag in final_replacement.keys():
            #tag is a major
            final_tags.append(tag)
        else:
            #tag is a minor
            r = []
            for k,v in final_replacement.items():
                if tag in v:
                    #tag is a minor w/ a major
                    r.append(k)
            if r == []:
                r.append(tag)
            final_tags += r
    final_tags = list(dict.fromkeys(final_tags))
    final_string = ','.join(final_tags)
    return final_string

In [19]:
df1 = df.copy()
df1['sim_tags'] = df1['tags']
df1['sim_tags'] = df1['sim_tags'].map(lambda x: replacing(x))
df1['sim_tags'] = df1['sim_tags'].map(lambda x: remove_tags(x,ted_tags))
df1 = df1[df1['sim_tags']!='']
df1

,speaker,headline,description,duration,tags,transcript,WC,clean_transcript,clean_transcript_string,sim_tags
0,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,0:16:17,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ...",2281.0,"[thank, chris, truly, great, honor, opportunit...",thank chris truly great honor opportunity come...,"cars,solar system,energy,culture,politics,scie..."
1,Amy Smith,Simple designs to save a life,Fumes from indoor cooking fires kill more than...,0:15:06,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ...",2687.0,"[term, invention, like, tell, tale, favorite, ...",term invention like tell tale favorite project...,"macarthur grant,simplicity,design,solar system..."
2,Ashraf Ghani,How to rebuild a broken state,Ashraf Ghani's passionate and powerful 10-minu...,0:18:45,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r...",2506.0,"[public, dewey, long, ago, observe, constitute...",public dewey long ago observe constitute discu...,"corruption,inequality,science,investment,war,c..."
3,Burt Rutan,The real future of space exploration,"In this passionate talk, legendary spacecraft ...",0:19:37,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst...",3092.0,"[want, start, say, houston, problem, enter, se...",want start say houston problem enter second ge...,"flight,design,nasa,science,invention,entrepren..."
4,Chris Bangle,Great cars are great art,American designer Chris Bangle explains his ph...,0:20:04,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac...",3781.0,"[want, talk, background, idea, car, art, actua...",want talk background idea car art actually mea...,"cars,design,transportation,invention,technolog..."
5,Craig Venter,Sampling the ocean's DNA,Genomics pioneer Craig Venter takes a break fr...,0:16:51,"biotech,invention,oceans,genetics,DNA,biology,...","0:11\r\r\rAt the break, I was asked by several...",2548.0,"[break, ask, people, comment, age, debate, com...",break ask people comment age debate comment un...,"biotech,invention,oceans,biomechanics,science,..."
6,David Pogue,Simplicity sells,New York Times columnist David Pogue takes aim...,0:21:26,"simplicity,computers,software,interface design...","0:12\r\r\r(Music: ""The Sound of Silence,""\rSim...",3584.0,"[music, sound, silence, simon, garfunkel, hell...",music sound silence simon garfunkel hello voic...,"simplicity,computers,software,design,music,med..."
7,David Rockwell,A memorial at Ground Zero,In this emotionally charged conversation with ...,0:24:37,"New York,memory,interview,death,culture,archit...","0:13\r\r\rKurt Andersen: Like many architects,...",4008.0,"[kurt, andersen, like, architect, david, hog, ...",kurt andersen like architect david hog limelig...,"new york,memory,interview,death,culture,archit..."
8,Dean Kamen,To invent is to give,Inventor Dean Kamen lays out his argument for ...,0:20:07,"robots,cars,industrial design,transportation,i...","0:11\r\r\rAs you pointed out, every time you c...",3278.0,"[point, time, come, learn, morning, world, exp...",point time come learn morning world expert gue...,"robots,cars,design,transportation,invention,sc..."
9,Dean Ornish,The killer American diet that's sweeping the p...,Forget the latest disease in the news: Cardiov...,0:03:18,"obesity,disease,health,health care,culture,foo...",0:11\r\r\rWith all the legitimate concerns\rab...,623.0,"[legitimate, concern, aid, avian, flu, hear, b...",legitimate concern aid avian flu hear brillian...,"science,culture,food,global issues"


In [20]:
tags1,tag1_list = count_tags(df1['sim_tags'])
tags1

science          1467
culture          1155
technology        787
global issues     679
design            477
history           385
business          349
entertainment     285
media             279
biomechanics      220
biodiversity      218
future            218
humanity          217
politics          199
communication     185
life              173
goal-setting      166
africa            166
collaboration     163
environment       155
brain             148
activism          147
music             143
invention         136
community         136
children          135
women             133
inequality        132
violence          122
animals           115
                 ... 
microfinance        4
vulnerability       4
glacier             4
3d printing         3
pandemic            3
forensics           3
mining              3
microsoft           3
sleep               3
brand               3
nobel prize         3
ptsd                3
trust               3
novel               2
origami   

In [21]:
tag_cutoff05 = int(0.05*len(df1.index))

squashed_tags05 = pd.DataFrame(tags1)
squashed_tags05 = squashed_tags05[(squashed_tags05[0]>tag_cutoff05)]
squash_list05 = list(squashed_tags05.index.values)
print(len(squash_list05))
squashed_tags05

29


,0
science,1467
culture,1155
technology,787
global issues,679
design,477
history,385
business,349
entertainment,285
media,279
biomechanics,220


In [22]:
df2 = df1.copy()
df2['squash29_tags'] = df2['sim_tags']
df2['squash29_tags'] = df2['squash29_tags'].map(lambda x: squashing(x,squash_list05))
df2 = df2[df2['squash29_tags']!='']
df2

,speaker,headline,description,duration,tags,transcript,WC,clean_transcript,clean_transcript_string,sim_tags,squash29_tags
0,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,0:16:17,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ...",2281.0,"[thank, chris, truly, great, honor, opportunit...",thank chris truly great honor opportunity come...,"cars,solar system,energy,culture,politics,scie...","culture,politics,science,global issues,environ..."
1,Amy Smith,Simple designs to save a life,Fumes from indoor cooking fires kill more than...,0:15:06,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ...",2687.0,"[term, invention, like, tell, tale, favorite, ...",term invention like tell tale favorite project...,"macarthur grant,simplicity,design,solar system...","design,invention,global issues"
2,Ashraf Ghani,How to rebuild a broken state,Ashraf Ghani's passionate and powerful 10-minu...,0:18:45,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r...",2506.0,"[public, dewey, long, ago, observe, constitute...",public dewey long ago observe constitute discu...,"corruption,inequality,science,investment,war,c...","inequality,science,culture,politics,global iss..."
3,Burt Rutan,The real future of space exploration,"In this passionate talk, legendary spacecraft ...",0:19:37,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst...",3092.0,"[want, start, say, houston, problem, enter, se...",want start say houston problem enter second ge...,"flight,design,nasa,science,invention,entrepren...","design,science,invention,business"
4,Chris Bangle,Great cars are great art,American designer Chris Bangle explains his ph...,0:20:04,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac...",3781.0,"[want, talk, background, idea, car, art, actua...",want talk background idea car art actually mea...,"cars,design,transportation,invention,technolog...","design,invention,technology,business,science"
5,Craig Venter,Sampling the ocean's DNA,Genomics pioneer Craig Venter takes a break fr...,0:16:51,"biotech,invention,oceans,genetics,DNA,biology,...","0:11\r\r\rAt the break, I was asked by several...",2548.0,"[break, ask, people, comment, age, debate, com...",break ask people comment age debate comment un...,"biotech,invention,oceans,biomechanics,science,...","invention,biomechanics,science,biodiversity,te..."
6,David Pogue,Simplicity sells,New York Times columnist David Pogue takes aim...,0:21:26,"simplicity,computers,software,interface design...","0:12\r\r\r(Music: ""The Sound of Silence,""\rSim...",3584.0,"[music, sound, silence, simon, garfunkel, hell...",music sound silence simon garfunkel hello voic...,"simplicity,computers,software,design,music,med...","design,music,media,entertainment,science,techn..."
7,David Rockwell,A memorial at Ground Zero,In this emotionally charged conversation with ...,0:24:37,"New York,memory,interview,death,culture,archit...","0:13\r\r\rKurt Andersen: Like many architects,...",4008.0,"[kurt, andersen, like, architect, david, hog, ...",kurt andersen like architect david hog limelig...,"new york,memory,interview,death,culture,archit...","culture,design,collaboration"
8,Dean Kamen,To invent is to give,Inventor Dean Kamen lays out his argument for ...,0:20:07,"robots,cars,industrial design,transportation,i...","0:11\r\r\rAs you pointed out, every time you c...",3278.0,"[point, time, come, learn, morning, world, exp...",point time come learn morning world expert gue...,"robots,cars,design,transportation,invention,sc...","design,invention,science,culture,technology,me..."
9,Dean Ornish,The killer American diet that's sweeping the p...,Forget the latest disease in the news: Cardiov...,0:03:18,"obesity,disease,health,health care,culture,foo...",0:11\r\r\rWith all the 

In [23]:
tags2,tag2_list = count_tags(df2['squash29_tags'])
tags2

science          1467
culture          1155
technology        787
global issues     679
design            477
history           385
business          349
entertainment     285
media             279
biomechanics      220
biodiversity      218
future            218
humanity          217
politics          199
communication     185
life              173
goal-setting      166
africa            166
collaboration     163
environment       155
brain             148
activism          147
music             143
invention         136
community         136
children          135
women             133
inequality        132
violence          122
dtype: int64

In [24]:
tag_cutoff075 = int(0.075*len(df1.index))

squashed_tags075 = pd.DataFrame(tags1)
squashed_tags075 = squashed_tags075[(squashed_tags075[0]>tag_cutoff075)]
squash_list075 = list(squashed_tags075.index.values)
print(len(squash_list075))
squashed_tags075

15


,0
science,1467
culture,1155
technology,787
global issues,679
design,477
history,385
business,349
entertainment,285
media,279
biomechanics,220


In [25]:
df3 = df1.copy()
df3['squash15_tags'] = df3['sim_tags']
df3['squash15_tags'] = df3['squash15_tags'].map(lambda x: squashing(x,squash_list075))
df3 = df3[df3['squash15_tags']!='']
df3

,speaker,headline,description,duration,tags,transcript,WC,clean_transcript,clean_transcript_string,sim_tags,squash15_tags
0,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,0:16:17,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ...",2281.0,"[thank, chris, truly, great, honor, opportunit...",thank chris truly great honor opportunity come...,"cars,solar system,energy,culture,politics,scie...","culture,politics,science,global issues,technology"
1,Amy Smith,Simple designs to save a life,Fumes from indoor cooking fires kill more than...,0:15:06,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ...",2687.0,"[term, invention, like, tell, tale, favorite, ...",term invention like tell tale favorite project...,"macarthur grant,simplicity,design,solar system...","design,global issues"
2,Ashraf Ghani,How to rebuild a broken state,Ashraf Ghani's passionate and powerful 10-minu...,0:18:45,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r...",2506.0,"[public, dewey, long, ago, observe, constitute...",public dewey long ago observe constitute discu...,"corruption,inequality,science,investment,war,c...","science,culture,politics,global issues,business"
3,Burt Rutan,The real future of space exploration,"In this passionate talk, legendary spacecraft ...",0:19:37,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst...",3092.0,"[want, start, say, houston, problem, enter, se...",want start say houston problem enter second ge...,"flight,design,nasa,science,invention,entrepren...","design,science,business"
4,Chris Bangle,Great cars are great art,American designer Chris Bangle explains his ph...,0:20:04,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac...",3781.0,"[want, talk, background, idea, car, art, actua...",want talk background idea car art actually mea...,"cars,design,transportation,invention,technolog...","design,technology,business,science"
5,Craig Venter,Sampling the ocean's DNA,Genomics pioneer Craig Venter takes a break fr...,0:16:51,"biotech,invention,oceans,genetics,DNA,biology,...","0:11\r\r\rAt the break, I was asked by several...",2548.0,"[break, ask, people, comment, age, debate, com...",break ask people comment age debate comment un...,"biotech,invention,oceans,biomechanics,science,...","biomechanics,science,biodiversity,technology"
6,David Pogue,Simplicity sells,New York Times columnist David Pogue takes aim...,0:21:26,"simplicity,computers,software,interface design...","0:12\r\r\r(Music: ""The Sound of Silence,""\rSim...",3584.0,"[music, sound, silence, simon, garfunkel, hell...",music sound silence simon garfunkel hello voic...,"simplicity,computers,software,design,music,med...","design,media,entertainment,science,technology"
7,David Rockwell,A memorial at Ground Zero,In this emotionally charged conversation with ...,0:24:37,"New York,memory,interview,death,culture,archit...","0:13\r\r\rKurt Andersen: Like many architects,...",4008.0,"[kurt, andersen, like, architect, david, hog, ...",kurt andersen like architect david hog limelig...,"new york,memory,interview,death,culture,archit...","culture,design"
8,Dean Kamen,To invent is to give,Inventor Dean Kamen lays out his argument for ...,0:20:07,"robots,cars,industrial design,transportation,i...","0:11\r\r\rAs you pointed out, every time you c...",3278.0,"[point, time, come, learn, morning, world, exp...",point time come learn morning world expert gue...,"robots,cars,design,transportation,invention,sc...","design,science,culture,technology,media,global..."
9,Dean Ornish,The killer American diet that's sweeping the p...,Forget the latest disease in the news: Cardiov...,0:03:18,"obesity,disease,health,health care,culture,foo...",0:11\r\r\rWith all the legitimate concerns\rab...,623.0,"[legitimate, concern,

In [26]:
tags3,tag3_list = count_tags(df3['squash15_tags'])
tags3

science          1467
culture          1155
technology        787
global issues     679
design            477
history           385
business          349
entertainment     285
media             279
biomechanics      220
future            218
biodiversity      218
humanity          217
politics          199
communication     185
dtype: int64

In [29]:
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
df3 = df3.reset_index(drop=True)

In [30]:
df2.to_parquet(OUT_DATA_DIR+'cleaned_squashed29_final.parquet')
df3.to_parquet(OUT_DATA_DIR+'cleaned_squashed15_final.parquet')